<h1> Get monthly data from postgres and write to json </h1>

In [8]:
from __future__ import division
import numpy as np
import json
import re
import gzip
import pandas as pd
import datetime
from ethiopian_date import ethiopian_date
import psycopg2

import gzip
from collections import OrderedDict

In [20]:
HEADERS = ['Real_Date', 'RegionName', 'ZoneName', 'WoredaName', 'WoredaLat', 'WoredaLon', \
    'source', 'field', 'val']
FIELDS = ['precipitation', 'humid_max', 'humid_min', 'humid_avg', 'solar', \
          'temp_max', 'temp_min', 'temp_avg', 'wind_avg', 'days_precipitation']

def get_geo_code(name_list):
    region, zone, woreda = name_list
    return ('%s__%s__%s__' % (region, zone, woreda)).lower()

def load_woreda_mapped(woreda_mapped_path):
    # Load useing dtype=str to prevent truncating the double
    woreda_df = pd.read_csv(woreda_mapped_path, dtype=str)
    # Remove columns used for name matching
    column_cut = [column for column in woreda_df.columns if 'match' not in column]
    woreda_df = woreda_df[column_cut]
    # Add GeoKey column
    woreda_df['GeoKey'] = [get_geo_code(name_list) for name_list in \
        woreda_df[['RegionName', 'ZoneName', 'WoredaName']].values]
    return woreda_df

def write_weather_to_gzjson(weather_df, woreda_mapped_path, output_dir):
    woreda_df = load_woreda_mapped(woreda_mapped_path)
    with gzip.open('%s/monthly_weather.json.gz' % output_dir, 'wr') as f:
        # Inner join the two dataframes on the GeoKey
        weather_merge = pd.merge(weather_df, woreda_df, how='inner', on='GeoKey')
        for row in weather_merge.iterrows():
            row = row[1]
            # Avoid pandas truncating the double
            lat = float(row['WoredaLat'])
            lon = float(row['WoredaLon'])
            woreda_consts = row[['Real_Date', 'RegionName', 'ZoneName', 'WoredaName']].tolist() \
                    + [lat, lon, 'weather']
            # Loop throgh all of the fields. Skip nan values.
            for field, value in row[FIELDS].dropna().iteritems():
                val_list = woreda_consts + [field, value]
                # OderedDict to perserve header order
                f.write(json.dumps(\
                    OrderedDict([(key, value) for key, value in zip(HEADERS, val_list)])))
                f.write('\n')

<h1> Pull in the monthly data from psql </h1>

In [21]:
conn = psycopg2.connect("host=localhost port=5432 dbname=et_weather user=attiladobi")
cur = conn.cursor()

sql_query = 'SELECT * FROM monthly;'
monthly_weather = pd.read_sql_query(sql_query, conn)
conn.close()

# Shift forward one month to be consistent with reporting at the start of the month on values of the previous
monthly_weather['e_date'] = [date + pd.offsets.relativedelta(months=1) for date in monthly_weather['e_date']]
monthly_weather['Real_Date'] = [ethiopian_date.EthiopianDateConverter.date_to_gregorian(date) \
                                for date in monthly_weather['e_date']]
monthly_weather['Real_Date'] = [date.strftime('%Y-%m-%d') for date in monthly_weather['Real_Date']]
monthly_weather.rename(columns={'geokey': 'GeoKey'}, inplace=True)
monthly_weather.head()

,e_date,GeoKey,precipitation,max_precipitation,days_precipitation,wind_avg,temp_max,temp_min,temp_avg,humid_max,humid_min,humid_avg,solar,Real_Date
0,2005-05-01 00:00:00-08:00,addis ababa__addis ababa__addis ketema__,4.646149,1.626706,8,2.931589,33.680000,8.46,24.125625,58.639999,0.00,20.821875,5411.228149,2013-01-09
1,2005-06-01 00:00:00-07:00,addis ababa__addis ababa__addis ketema__,13.285122,1.769140,23,3.552899,35.830002,10.84,21.191000,80.169998,0.00,38.482334,5417.762174,2013-02-08
2,2005-07-01 00:00:00-07:00,addis ababa__addis ababa__addis ketema__,11.752901,1.446901,24,4.583223,36.830002,9.30,22.872333,79.059998,0.00,33.474667,6313.152474,2013-03-10
3,2005-08-01 00:00:00-07:00,addis ababa__addis ababa__addis ketema__,26.476681,3.206667,28,4.199224,36.610001,12.08,24.810333,94.959999,5.46,42.719167,4710.198372,2013-04-09
4,2005-09-01 00:00:00-07:00,addis ababa__addis ababa__addis ketema__,42.479760,26.730000,22,4.172926,35.439999,14.75,24.162167,100.000000,6.07,47.695667,4752.063078,2013-05-09


<h1> Main </h1>

In [22]:
woreda_mapped_path = '/Users/attiladobi/zenysis/pipeline/auto/ethiopia/bin/shared/data/woreda_mapped.csv'
write_weather_to_gzjson(monthly_weather, woreda_mapped_path, 'monthly_weather')